In [1]:
%cd ..
import os, sys
sys.path.append(os.path.join(os.getcwd(), "src"))

d:\Blockchain\data\orai\orchai_validator


In [2]:
from orchai.upload import ETLProcessor, get_spark, sampling, uploading, validating
from omegaconf import OmegaConf
import pyspark.sql.functions as F
import numpy as np

In [3]:
spark = get_spark()
config = OmegaConf.load("config/etl_file_1m.yaml")
sb, eb = 7059473, 7659473
print((eb - sb) / 150 + 1)
df = sampling(spark, config, sb, eb)
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

4001.0
------------------------------------------------
start-end final blocks: 7059473 7659473
Successfully converted voting_power_score column
------------------------------------------------
start-end final blocks: 7059473 7659473
Successfully converted commission_score column
------------------------------------------------
start-end final blocks: 7059473 7659473
Successfully converted self_bonded_score column
------------------------------------------------
start-end final blocks: 7059473 7659473
Successfully converted vote_propose_score column
------------------------------------------------
start-end final blocks: 7073723 7659473
error block: 7272623 7273073 450
Sucessfully converted final_score
------------------------------------------------
start-end final blocks: 7073723 7659473
error block: 7272623 7273073 450
Sucessfully shifting data
------------------------------------------------
start-end final blocks: 7073723 7227173
start-end final blocks: 7073723 7227173


In [5]:
7073723 + 14400

7088123

In [7]:
14400 / 150

96.0

In [11]:
def validating(dff):
    import numpy as np
    block_heights = dff.groupBy("block_height").count().select(F.col("block_height"))
    block_heights = block_heights.toPandas().to_numpy().reshape((-1,))
    block_heights.sort()
    print("start-end final blocks:", block_heights[0], block_heights[-1])

    diff = np.diff(block_heights)
    for i, d in enumerate(diff):
        if d != 150:
            print("error block:", block_heights[i], block_heights[i + 1], d)

start-end final blocks: 7073723 7227173


In [13]:
7659473 - 7227173

432300

In [14]:
# config = OmegaConf.load("config/local_file.yaml")
# sb, eb = 7059473, 7659473
# print((eb - sb) / 150 + 1)
# df = sampling(spark, config, sb, eb)
# dff = ETLProcessor.data_scoring(df, **config.hp.etl)

4001.0
------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [15]:
# block_heights = dff.groupBy("block_height").count().select(F.col("block_height"))
# block_heights = block_heights.toPandas().to_numpy().reshape((-1,))
# block_heights.sort()
# print("start-end final blocks:", block_heights[0], block_heights[-1])

# diff = np.diff(block_heights)
# for i, d in enumerate(diff):
#     if d != 150:
#         print("error block:", block_heights[i], block_heights[i + 1], d)

start-end final blocks: 7059923 7658273
error block: 7272623 7273073 450


In [16]:
7659473 - 7658273

1200